In [1]:
# !pip install datasets transformers

In [2]:
import torch
device = 'cuda:5' if torch.cuda.is_available() else 'cpu'
device

'cuda:5'

In [3]:
from collections import defaultdict
import os
import datetime
from pathlib import Path
import json
from tqdm import tqdm

from torch import nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler

# import datasets
# from datasets import load_dataset
from transformers import AutoTokenizer

import utils.load_corpus as load_corpus
import utils.losses as l

import utils.RNN as RNN
import model.absolute_bert as Abs_bert

import importlib

In [4]:
PATH = "data"
folder_path = Path(f"limanet")
os.makedirs(PATH/folder_path, exist_ok=True)

In [5]:
tokenizer_type = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
text = ["I love bird.", "Hi, I'm Bob."]
tokenized = tokenizer(text, return_tensors='pt', padding=True)
tokenized['input_ids']

tensor([[ 101, 1045, 2293, 4743, 1012,  102,    0,    0,    0],
        [ 101, 7632, 1010, 1045, 1005, 1049, 3960, 1012,  102]])

In [6]:
# def transform(batch):
#   tokenized = tokenizer(batch['text'], return_tensors='pt', padding=True, add_special_tokens=False)
#   packed = torch.nn.utils.rnn.pack_padded_sequence(tokenized['input_ids'], tokenized['attention_mask'].sum(dim=1), enforce_sorted=False, batch_first=True)
#   to_return = {
#     'data': packed.data,
#     'batch_sizes': packed.batch_sizes,
#     'attention_mask': tokenized['attention_mask'],
#     'sorted_indices': packed.sorted_indices,
#     'unsorted_indices': packed.unsorted_indices
#   }
#   return to_return

# dataset.set_transform(transform)

# dataset['train'][:2]

In [7]:
# # test
# rotator_depth = 3

# import importlib
# rotator = importlib.reload(rotator)
# RNN = importlib.reload(RNN)

# model = RNN.RNN(rotator.Rotator(tokenizer.vocab_size, depth=rotator_depth, dim=128, num_heads=32, hippo_dim=16, num_hippo_heads=8))#.to(device)
# model.eval()
# batch = dataset['train'][:3]

# gen = model(**batch)
# predicts, targets = [torch.concat(res) for res in zip(*gen)]
# predicts.shape, targets.shape


In [8]:
%%time

batch_size = 128
masking_probability = .3
shuffle = True

import importlib
load_corpus = importlib.reload(load_corpus)

dataset = load_corpus.Load_bookcorpus(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
# dataset = load_corpus.load_msmarco(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')
dataset = load_corpus.Load_wiki(tokenizer, batch_size=batch_size, shuffle=shuffle, cache_dir='~/data1-0756727/cache/huggingface')

dataset.transformer(masking_probability=masking_probability)
print(dataset.dataset['train'][:2])

train_loader = dataset.loader()
n = next(iter(train_loader))
# print(tokenizer.decode(n['data']))
{key: n[key].shape for key in n}

/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for bookcorpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bookcorpus
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'input_ids': tensor([[ 9617,   103,   103,  ...,  2163,   103,   103],
        [ 1010,  2030, 23560,  ..., 11140,  2964,  1012],
        [ 1999,   103,  2197,  ...,  2008,  2009,  2003],
        ...,
        [29279,   103, 15161,  ...,   103,  1996,  2451],
        [ 1012, 14260,  1010,  ...,  2012,   103,   103],
        [  103,   103,   103,  ...,  1010,   103,  2500]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ -100, 11140,  2964,  ...,  -100,  1010, 18814],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  1996,  -100,  ...,  -100,  -100,  -100],
        ...,
        [ 1012, 25962, 15161,  ...,  1999,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100, 

{'input_ids': torch.Size([342, 128]),
 'token_type_ids': torch.Size([128, 512]),
 'attention_mask': torch.Size([342, 128]),
 'labels': torch.Size([342, 128])}

In [9]:
# from transformers import DataCollatorForLanguageModeling

# class Get_transform:

#   def __init__(self, masking_probability=0.15, max_length=128, num_chunks=4):
    
#     self.max_length = max_length
#     self.num_chunks = num_chunks

#   def get_transform(self):
#     def transform(batch):
#       tokenized = tokenizer(batch['text'],
#         return_tensors='pt',
#         padding=True,
#         add_special_tokens=False,
#         max_length=self.max_length*self.num_chunks,
#         truncation=True)
#       input_ids = tokenized['input_ids'].view(-1, self.max_length)
#       attention_mask = tokenized['attention_mask'].view(-1, self.max_length)
#       lengths = attention_mask.sum(dim=1)
#       indices = ~(lengths == 0)
  
#       input_ids, labels = self.collator.torch_mask_tokens(input_ids[indices])
      
#       return tokenized | {'input_ids': input_ids, 'attention_mask': attention_mask[indices], 'labels': labels}
#     return transform

# get_transform = Get_transform(masking_probability, max_length=256, num_chunks=4)
# dataset.dataset.set_transform(get_transform.get_transform())

In [10]:
import numpy as np

class Sampled_softmax_cross_entropy(nn.Module):
  """https://douglasorr.github.io/2021-10-training-objectives/3-sampled/article.html"""
  
  def __init__(self, model, num_sampling=100):
    super().__init__()
    self.num_sampling = num_sampling
    self.model = model

  def forward(self, predictions, labels):
    """
    preds: shape [batch_size, dim]
    labels: shape [batch_size, dim]
    """
    # batch_sizes = predictions.shape[:-1]
    
    # model = ...  # returns (batch_size x embedding_size)
    projection = self.model.word_embeddings()  # shape (n_classes x embedding_size)
    n_classes = projection.shape[0]
    
    # 2. Get target label scores, paired_inner_product(pred_emb, label_emb)
    label_scores = (predictions * projection[labels, :]).sum(-1) + self.model.bias[labels]
    
    # 3. Sample shared noise & get scores
    samples = torch.randint(high=n_classes, size=[self.num_sampling]).to(labels.device)
    noise_scores = predictions @ projection[samples, :].T + self.model.bias[None, samples]
    noise_scores += np.log(n_classes - 1)
    
    # 4. Reject samples matching target label & correct for remaining samples
    reject_samples = (labels[..., None] == samples[None, :]) & (labels[..., None] != -100)  #後面是 collator 會把非預測目標填為 -100
    noise_scores -= 1e6 * reject_samples
    noise_scores -= torch.log((self.num_sampling - reject_samples.sum(-1, keepdims=True)).float())
    
    # 5. Apply regular softmax cross entropy
    scores = torch.cat([label_scores[..., None], noise_scores], dim=-1)
    pseudo_label = torch.masked_fill(labels.clone(), labels != -100, 0).view(-1)
    loss = torch.nn.functional.cross_entropy(scores.view(-1, scores.shape[-1]), pseudo_label)

    return loss

In [11]:
learning_rate = 0.01 # for cross entropy
learning_rate = 0.001 # for fine tuning

model_params = {
  'depth': 8,
  'num_heads': 8,
  'dim': 512,
  # 'hidden_dim': 32,
  # 'embedding_initialize_method': 'rand'
}
loss_params = {
  'sampling_word_size': 100,
}

Abs_bert = importlib.reload(Abs_bert)
l = importlib.reload(l)

# model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params), weight_tying=False).to(device)
# using_loss = l.Complex_mse()
# using_loss = l.Complex_triplet_loss(model)
# using_loss = l.Complex_mse_triplet_loss(model)
# using_loss = l.Complex_mse_squared_triplet_loss(model)
# using_loss = l.Mse_multiplet_loss(model.model, **loss_params)
# using_loss = l.Cos_multiplet_loss(model.model, **loss_params)

model = Abs_bert.Absolute_bert_for_masked_LM(tokenizer.vocab_size, **model_params).to(device)
# using_loss = l.Sampled_softmax_cross_entropy(model, num_sampling=loss_params['sampling_word_size'])
using_loss = Sampled_softmax_cross_entropy(model, num_sampling=loss_params['sampling_word_size'])

# model = RNN_base(rotator.Rotator(tokenizer.vocab_size, **model_params)).to(device)
# using_loss = nn.CrossEntropyLoss()
# using_loss = l.Cross_entropy_l2_embedding(model.model, min_squared_norm=model.model.dim)
# using_loss = l.Cross_entropy_l2_embedding(model.model, l2_squared_weight=1e-3)

# subpath = '20240624.12:08:14' # 4 ssm_ce_with_bias
# batch_num = 25000
# model = torch.load(PATH/folder_path/subpath/f'batch_{batch_num}-model.pt', map_location=device)

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, total_iters=10, start_factor=0.01)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)

In [12]:
# #test
# batch = next(iter(train_loader))
# batch_cuda = {k: v.to(device) for k, v in batch.items()}
# res = model(**batch_cuda)

# # using_loss = Sampled_softmax_cross_entropy(model, num_sampling=loss_params['sampling_word_size'])
# # l_ = using_loss(res[0], res[1])

# preds = res[0] @ model.word_embeddings().T
# l_ = nn.functional.cross_entropy(preds.view(-1, tokenizer.vocab_size), res[1].view(-1))

In [13]:
# # testing model output
# batch = next(iter(train_loader))
# batch = {key: batch[key].to(device) for key in batch}

# gen = model(**batch)
# predicts, targets = [torch.concat(res) for res in zip(*gen)]
# losses = using_loss(predicts, targets)
# loss = sum(losses.values())
# loss_dict = {
#   'total': f"{loss.item(): .6f}",
#   **{key: f"{losses[key].item(): .6f}" for key in losses},
# }

# print(loss_dict)
# # del batch, loss
# # torch.cuda.empty_cache()

In [14]:
# def backward_hook(module, gin, gout):
#   print(f"{len(gin)=}, {len(gout)=}")
#   print(*[f"{i=}, {gi.shape=}, {gi.mean()=}, {gi.min()=}, {gi.max()=}" for i, gi in enumerate(gin)], sep='\n')
#   print(*[f"{i=}, {go.shape=}, {go.mean()=}, {go.min()=}, {go.max()=}" for i, go in enumerate(gout)], sep='\n')

# model.limas[0].register_backward_hook(backward_hook)

In [15]:
time_string = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=8))).strftime('%Y%m%d.%H:%M:%S')
subfolder_path = Path(time_string)

os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

with open(PATH/folder_path/subfolder_path/f'parameters.json', 'w') as f:
  f.write(json.dumps({
    'tokenizer_type': tokenizer_type,
    'model': str(model),
    'model_params': model_params,
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'masking_probability': masking_probability,
    'shuffle': shuffle,
    'loss_type': str(using_loss),
    'loss_params': loss_params,
  }))  
time_string

'20240704.21:19:36'

In [ ]:
num_epochs = 3
save_every_n_batches = 5000

multiloss = 0

# os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

loss_history = []

for epoch_num, epoch in enumerate(range(num_epochs)):
  bar = tqdm(train_loader)

  for batch_num, batch in enumerate(bar):

    optimizer.zero_grad()
    
    batch = {key: batch[key].to(device) for key in batch}
    predicts, targets = model(**batch)

    # predicts = predicts @ model.word_embeddings().T + model.bias
    # predicts = predicts.view(-1, tokenizer.vocab_size)
    # targets = targets.view(-1)
    
    if multiloss:
      losses = using_loss(predicts, targets)
      loss = sum(losses.values())
    else:
      loss = using_loss(predicts, targets)
    
    loss_dict = {
      'loss': f"{loss.item(): .6f}",
    }
    if multiloss:
      loss_dict |= {key: f"{losses[key].item(): .6f}" for key in losses}
      
    bar.set_postfix(loss_dict)

    with torch.autograd.detect_anomaly(True):
      loss.backward()
      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=10, norm_type=2)
    # loss.backward()
    # break 
    optimizer.step()

    if (batch_num % 100 == 0):
      # torch.cuda.empty_cache()
      loss_history.append([epoch_num, batch_num, loss_dict])
      # print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, batch {batch_num}")

    if (batch_num % 1000 == 999):
      scheduler.step()

    if (batch_num % save_every_n_batches == 0):
      prefix = f"epoch_{epoch_num}-batch_{batch_num}-"
      torch.save(model, PATH/folder_path/subfolder_path/(prefix + f'model.pt'))
        
      with open(PATH/folder_path/subfolder_path/f'epoch_{eopch_num}-history.json', 'w') as f:
        f.write(json.dumps(loss_history))  

      # torch.cuda.empty_cache()

    if (batch_num in [500, 1000, 1500, 2000, 2500, 5000, 7500, 12500, 17500]):
        prefix = f"epoch_{epoch_num}-batch_{batch_num}-"
        torch.save(model, PATH/folder_path/subfolder_path/(prefix + 'model.pt'))
        
        with open(PATH/folder_path/subfolder_path/f'epoch_{epoch_num}-history.json', 'w') as f:
          f.write(json.dumps(loss_history))  

    # del batch, loss

  0%|          | 0/50459 [00:00<?, ?it/s, loss=43.076263]/tmp/ipykernel_56272/1220748506.py:38: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly(True):
  5%|▌         | 2735/50459 [24:41<6:17:16,  2.11it/s, loss=11.534546] 

In [ ]:
inputs = tokenizer('I love you.', return_tensors='pt', add_special_tokens=False)
histories = [his.expand(4, *[-1]*(len(his.shape)-1)) for his in model.model.initial_states()['h']]
output, next_histories = model.model.forward(inputs['input_ids'][0].to(device), histories)

In [ ]:
output